In [203]:
import os

from cassandra.cluster import Cluster
from cassandra.query import BatchStatement
from cassandra.auth import PlainTextAuthProvider
from cassandra import WriteTimeout

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as _sum


import seaborn as sns
import matplotlib.pyplot as plt

import csv

import numpy as np
from scipy.stats import linregress
from scipy.optimize import minimize

import pandas as pd

import yfinance as yf
from datetime import datetime, timedelta
import time
from tqdm import tqdm

In [2]:
# Load ticker symbols from nasdaq screener csv file
def load_tickers(filepath):
    df = pd.read_csv(filepath)
    return df.iloc[:, 0].tolist()

In [3]:
# Fetch historical data for a given ticker
def fetch_stock_data(ticker):
    # Calculate the date 5 years ago from today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*5)
    
    # Fetch historical data
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return data

In [ ]:
# Load ticker symbols
tickers = load_tickers('nasdaq_screener.csv')

# Create a subfolder for the financial data
output_dir = 'fin_data'
os.makedirs(output_dir, exist_ok=True)

# Dictionary to hold data for all tickers
all_data = {}

# Loop through each ticker and fetch its data
index = 0
for ticker in tickers:
    try:
        data = fetch_stock_data(ticker)
        all_data[ticker] = data
        # Save data
        data.to_csv(os.path.join(output_dir, f'{ticker}_5_years_data.csv'))
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

In [ ]:
# Define the directory containing the CSV files
source_dir = os.path.expanduser('fin_data/')

# List all files in the directory
files = os.listdir(source_dir)

# Loop through each file
for file in files:
    file_path = os.path.join(source_dir, file)
    
    # Check if the file is a CSV file
    if file.endswith('.csv'):
        try:
            # Load the CSV file
            df = pd.read_csv(file_path)
            
            # Check if the file is empty or contains NA values
            if df.empty or df.isna().any().any():
                os.remove(file_path)  # Remove the file if it's empty or contains NA
                print(f"Deleted '{file}' because it was empty or contained NA values.")
        except Exception as e:
            print(f"Error processing {file}: {e}")

print("Cleanup complete.")


In [275]:
def create_schema(session):
    # CQL to create a keyspace
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS fin_data WITH replication = {
            'class': 'SimpleStrategy', 
            'replication_factor': 1
        }
    """)

    # CQL to use the keyspace
    session.execute("USE fin_data")

    # CQL to create a table
    session.execute("""
        CREATE TABLE IF NOT EXISTS ohlcv (
            symbol text,
            date timestamp,
            open double,
            high double,
            low double,
            close double,
            adj_close double,
            volume bigint,
            PRIMARY KEY (symbol, date)
        )
    """)

In [276]:
def parse_date(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d')

In [277]:
def execute_with_retries(session, batch, max_retries=5):
    retries = 0
    backoff_time = 2  # Start with 2 seconds backoff time
    while retries < max_retries:
        try:
            session.execute(batch)
            break  # Break the loop if execution is successful
        except WriteTimeout:
            print(f"WriteTimeout: retrying {retries+1}/{max_retries} after {backoff_time}s...")
            time.sleep(backoff_time)  # Sleep for backoff_time seconds before retrying
            backoff_time *= 2  # Double the backoff time for the next retry
            retries += 1
    if retries == max_retries:
        print("Failed to execute batch after several retries.")

In [278]:
def load_data(session, file_path, symbol):
    batch = BatchStatement()
    batch_size = 0  # Track the size of the batch

    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            batch.add(
                """
                INSERT INTO fin_data.ohlcv (symbol, date, open, high, low, close, adj_close, volume)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (symbol, parse_date(row['Date']), float(row['Open']), float(row['High']),
                 float(row['Low']), float(row['Close']), float(row['Adj Close']), int(row['Volume']))
            )
            batch_size += 1
            if batch_size >= 100:  # Execute batch after collecting 50 statements
                execute_with_retries(session, batch)
                batch = BatchStatement()  # Reset batch after execution
                batch_size = 0  # Reset batch size

        if batch_size > 0:  # Ensure any remaining statements are executed
            execute_with_retries(session, batch)

        print(f"Data from {symbol} loaded successfully.")

In [279]:
def drop_table(keyspace_name, table_name):
    """ Drop a table in a given keyspace """
    # Set up connection parameters
    auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')  # Update credentials if needed
    cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)  # Adjust IP if Cassandra is hosted elsewhere
    session = cluster.connect(keyspace_name)  # Connect to the specified keyspace

    # Drop table
    try:
        session.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table {table_name} has been dropped successfully.")
    except Exception as e:
        print(f"An error occurred while dropping the table: {e}")
    finally:
        # Clean up, close the session and cluster connection
        session.shutdown()
        cluster.shutdown()

In [280]:
def query_symbol_data(symbol, start_date, end_date):
    
    cluster = Cluster(['127.0.0.1'])  # Adjust if Cassandra is hosted elsewhere
    session = cluster.connect('fin_data')

    # Prepare the query
    query = """
        SELECT * FROM ohlcv WHERE symbol = %s AND date >= %s AND date <= %s;
    """
    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    # Execute the query
    try:
        rows = session.execute(query, (symbol, start_dt, end_dt))
        for row in rows:
            print(row)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Clean up, close the session and cluster connection
        session.shutdown()
        cluster.shutdown()

In [281]:
def create_table():
    # Connect to Cassandra
    # Adjust the connection settings as needed for your setup
    auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')  # Update with actual credentials
    cluster = Cluster(['127.0.0.1'], auth_provider=auth_provider)
    session = cluster.connect()

    # Create the schema
    create_schema(session)

    print("Keyspace and table created successfully.")

    # Clean up
    session.shutdown()
    cluster.shutdown()

In [282]:
create_table()

Keyspace and table created successfully.


In [284]:
cluster = Cluster(['127.0.0.1'])  # Adjust if Cassandra is hosted elsewhere
session = cluster.connect()

i = 0
data_dir = '/home/xca7/Desktop/Storage-Solutions-for-Big-Data-CA1/fin_data'
for filename in tqdm(os.listdir(data_dir)):
    if i > 1000:
        break
    i += 1
    if filename.endswith('.csv'):
        symbol = filename.split('_')[0]  # Extract the symbol from the filename
        load_data(session, os.path.join(data_dir, filename), symbol)

session.shutdown()
cluster.shutdown()

  0%|          | 1/6638 [00:00<1:38:54,  1.12it/s]

Data from PEBK loaded successfully.


  0%|          | 2/6638 [00:01<1:42:39,  1.08it/s]

Data from ARR loaded successfully.


  0%|          | 3/6638 [00:02<1:16:44,  1.44it/s]

...

Data from IMTX loaded successfully.


 15%|█▌        | 1000/6638 [14:10<1:11:18,  1.32it/s]

Data from KIDS loaded successfully.


 15%|█▌        | 1001/6638 [14:11<1:19:54,  1.18it/s]

Data from BB loaded successfully.


In [55]:
#drop_table('fin_data', 'ohlcv')

Table ohlcv has been dropped successfully.


In [285]:
def create_spark_session():
    spark = SparkSession.builder \
        .appName("Cassandra Integration Example") \
        .config("spark.cassandra.connection.host", "localhost") \
        .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
        .getOrCreate()
    return spark

In [286]:
# Initialize Spark session
spark = create_spark_session()

In [287]:
def list_unique_symbols(spark, keyspace, table):
    # Load the data from Cassandra into a DataFrame
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace=keyspace, table=table) \
        .load()
    
    # Select distinct symbols and collect them into a list
    symbols = df.select("symbol").distinct().collect()
    
    # Extract symbols from rows
    symbol_list = [row['symbol'] for row in symbols]
    print("Unique symbols:", symbol_list)
    return symbol_list

In [288]:
keyspace = 'fin_data'
table = 'ohlcv'
symbols = list_unique_symbols(spark, keyspace, table)

Unique symbols: ['GGN', 'AZ', 'AHCO', 'AMRK', 'RANI', 'BW', 'SILK', 'BZFD', 'GFR', 'EOSEW', 'CTGO', 'RMR', 'REGCO', 'DBGI', 'NVMI', 'KREF', 'STBX', 'SCPX', 'DUET', 'CFSB', 'MDAI', 'DTIL', 'EQ', 'XP', 'POST', 'NOG', 'ELA', 'LTRX', 'YTEN', 'MBUU', 'HTH', 'DMA', 'WVVIP', 'LGCL', 'CYBN', 'ACONW', 'CIFRW', 'IVVD', 'ASCB', 'PHG', 'UBSI', 'NITO', 'NWBI', 'DYCQ', 'IINNW', 'TNON', 'DOMO', 'NOK', 'PEO', 'CIVB', 'PFGC', 'NRIM', 'FANH', 'CBAT', 'NXPL', 'LAD', 'ACST', 'NTNX', 'OMI', 'GASS', 'MAR', 'X', 'THO', 'GMFIW', 'ICE', 'ACMR', 'CGNX', 'IMNM', 'ACRS', 'BIPC', 'NUKK', 'AMBP', 'MHI', 'ALVO', 'CLCO', 'ARR', 'GNTX', 'OPAL', 'PARR', 'PEBK', 'PLUR', 'COE', 'SYM', 'LFUS', 'KIDS', 'GBCI', 'SOJC', 'HYFM', 'SUPN', 'OSG', 'FLWS', 'TFX', 'ATS', 'PAAS', 'IMXI', 'BFRGW', 'TAYD', 'LENZ', 'ELME', 'SHOT', 'SBET', 'TMHC', 'PETS', 'EURN', 'FFIV', 'SNN', 'AMK', 'MNTX', 'FULTP', 'HTIBP', 'ARE', 'ELS', 'TOL', 'ANTE', 'SEAT', 'FORA', 'BRLSW', 'DCOMP', 'CRIS', 'AFGD', 'GRTS', 'BITF', 'BLZE', 'GLO', 'XPO', 'IR', 'VCYT

In [289]:
def query_symbol_data(spark, keyspace, table, symbol, start_date, end_date):
    # Load data from the specified table and keyspace
    df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(keyspace=keyspace, table=table) \
        .load()

    # Filter data for the specified symbol and date range
    filtered_df = df.filter(
        (col("symbol") == symbol) & 
        (col("date") >= start_date) & 
        (col("date") <= end_date)
    )

    # Convert to Pandas DataFrame
    pandas_df = filtered_df.toPandas()
    return pandas_df

In [290]:
# Example usage
keyspace = 'fin_data'
table = 'ohlcv'
symbol = 'PEBK'  # Example symbol
start_date = '2019-01-01'  # Example start date
end_date = '2024-01-31'  # Example end date
    
query_symbol_data(spark, keyspace, table, symbol, start_date, end_date)


,symbol,date,adj_close,close,high,low,open,volume
0,PEBK,2019-04-23 01:00:00,25.188534,27.450001,27.450001,27.309999,27.309999,1600
1,PEBK,2019-04-24 01:00:00,24.885725,27.120001,27.459999,26.750000,27.450001,2400
2,PEBK,2019-04-25 01:00:00,24.968306,27.209999,27.750000,27.209999,27.750000,2300
3,PEBK,2019-04-26 01:00:00,24.922428,27.160000,27.790001,26.900000,27.309999,138100
4,PEBK,2019-04-29 01:00:00,25.142654,27.400000,27.730000,27.070000,27.070000,18200
...,...,...,...,...,...,...,...,...
1198,PEBK,2024-01-25 00:00:00,29.670000,29.670000,29.750000,29.540001,29.650000,3200
1199,PEBK,2024-01-26 00:00:00,29.799999,29.799999,29.959999,29.799999,29.959999,3700
1200,PEBK,2024-01-29 00:00:00,29.670000,29.670000,29.799999,29.500000,29.590000,6600
1201,PEBK,2024-01-30 00:00:00,29.570000,29.570000,29.879999,29.570000,29.610001,1600


In [ ]:
# Lets construct some portfolios and see how they perform!

In [88]:
def trend_analysis(data):
    """
    Perform linear regression on the closing prices to determine the trend.
    """
    y = data['close']
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return {'slope': slope, 'intercept': intercept, 'r_value': r_value}


In [89]:
def volatility_analysis(data):
    """
    Calculate the annualized volatility of stock prices.
    """
    returns = data['close'].pct_change()
    volatility = returns.std() * np.sqrt(252)  # Assuming 252 trading days in a year
    return volatility


In [128]:
def moving_averages(data, window=20):
    """
    Calculate the simple moving average (SMA) and exponential moving average (EMA).
    """
    sma = data['close'].rolling(window=window).mean()
    ema = data['close'].ewm(span=window, adjust=False).mean()
    return sma, ema


In [151]:
def test_cyclicity(data, threshold=0.05):
    
    n = len(data)
    freqs = np.fft.fftfreq(n)
    fft_values = np.fft.fft(data['close'])
    amplitudes = np.abs(fft_values) / n

    # Check if there are any significant amplitudes (ignoring the zero frequency)
    significant_freqs = (freqs > 0) & (amplitudes > threshold)
    return np.any(significant_freqs)


In [137]:
from statsmodels.tsa.seasonal import seasonal_decompose

def test_seasonality(data, model='additive', freq=None):
    result = seasonal_decompose(data, model=model, period=freq)
    return np.any(np.abs(result.seasonal) > 0.01)


In [173]:
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_seasonal_decompose(data, model='additive', freq=365):
    result = seasonal_decompose(data['close'], model=model, period=freq)
    result.plot()
    plt.show()


In [174]:
def plot_rolling_statistics(data, window=30):
    rolling_mean = data['close'].rolling(window=window).mean()
    rolling_std = data['close'].rolling(window=window).std()

    plt.figure(figsize=(8, 6))
    plt.plot(data['close'], color='blue', label='Original')
    plt.plot(rolling_mean, color='red', label='Rolling Mean')
    plt.plot(rolling_std, color='black', label='Rolling Std Deviation')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()


In [169]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_autocorrelation(data):
    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plot_acf(data['close'], ax=plt.gca())
    plt.title('Autocorrelation Function')

    plt.subplot(122)
    plot_pacf(data['close'], ax=plt.gca())
    plt.title('Partial Autocorrelation Function')
    
    plt.tight_layout()
    plt.show()


In [197]:
def plot_correlation_matrix(all_data):
    # Extract close prices and create a new DataFrame
    close_prices = pd.DataFrame()
    
    for symbol, df in all_data.items():
        if 'close' in df.columns:
            close_prices[f'{symbol}'] = df['close']
        else:
            print(f"Warning: 'close' column not found in DataFrame index {i}")

    # Calculate the correlation matrix
    correlation_matrix = close_prices.corr()
    
    # Plotting the correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title('Correlation Matrix of Stock Close Prices')
    plt.show()



In [294]:
all_data = {}
for symbol in tqdm(symbols):
    all_data[symbol] = query_symbol_data(spark, keyspace, table, symbol, start_date, end_date)

  0%|          | 0/1001 [00:00<?, ?it/s]

100%|██████████| 1001/1001 [03:34<00:00,  4.67it/s]


In [244]:
def diversification_through_correlation(stock_data, threshold=0.5):
    
    # Extract 'Close' prices and consolidate into a single DataFrame
    close_prices = pd.DataFrame({
        symbol: data['close'] for symbol, data in stock_data.items()
        if 'close' in data.columns
    })

    # Handle the case where no valid 'Close' data is found
    if close_prices.empty:
        print("No valid 'close' data found in the input dictionaries.")
        return []

    # Calculate the correlation matrix
    corr_matrix = close_prices.corr()

    # Calculate average correlation for each stock
    avg_correlations = corr_matrix.mean().sort_values()

    # Filter stocks based on the threshold and ensure correlations are not negative
    low_corr_stocks = avg_correlations[(-threshold < avg_correlations) & (avg_correlations < threshold)].index.tolist()

    return low_corr_stocks


In [295]:
import pandas as pd
import numpy as np

def risk_and_return_analysis(stock_data, risk_free_rate=0.1):
    
    returns_data = {}

    for symbol, data in stock_data.items():
        if 'close' in data.columns:
            # Calculate daily returns
            daily_returns = data['close'].pct_change().dropna()
            # Annualize returns
            annual_returns = np.mean(daily_returns) * 252  # assuming 252 trading days in a year
            # Annualize volatility
            annual_volatility = np.std(daily_returns) * np.sqrt(252)

            # Calculate Sharpe ratio
            sharpe_ratio = (annual_returns - risk_free_rate) / annual_volatility

            returns_data[symbol] = {
                'Return': annual_returns,
                'Volatility': annual_volatility,
                'Sharpe Ratio': sharpe_ratio
            }
        else:
            print(f"Warning: 'close' column not found in DataFrame for {symbol}")

    # Compile results into a DataFrame
    summary_df = pd.DataFrame(returns_data).T  # Transpose to make symbols as rows

    # Filter to only include stocks with a Sharpe ratio > 1
    filtered_df = summary_df[summary_df['Sharpe Ratio'] > 1]

    return filtered_df


In [301]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

def optimize_portfolio(stock_data, target_return, risk_free_rate=0.01):
    # Extract 'Close' prices and calculate returns
    returns = pd.DataFrame({
        symbol: data['close'].pct_change() for symbol, data in stock_data.items() if 'close' in data.columns
    }).dropna()

    # Calculate the covariance matrix
    covariance_matrix = returns.cov()
    num_assets = len(returns.columns)

    # Constraints (weights sum to 1 and achieve target return)
    constraints = [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # weights must sum to 1
        {'type': 'eq', 'fun': lambda weights: returns.mul(weights).sum().mean() * 252 - target_return}  # target return
    ]
    bounds = tuple((0, 1) for _ in range(num_assets))

    # Objective function (portfolio variance)
    def portfolio_variance(weights):
        return weights.T @ covariance_matrix @ weights

    # Initial guess (equal distribution)
    initial_weights = num_assets * [1. / num_assets]

    # Minimize the portfolio variance
    result = minimize(portfolio_variance, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

    # Return a dictionary of stock symbols and their weights if optimization is successful
    if result.success:
        optimized_weights = dict(zip(returns.columns, result.x))
        return optimized_weights
    else:
        print("Optimization failed:", result.message)
        return {}



In [348]:
def construct_and_plot_portfolio(stock_data, weights):
    
    portfolio_time_series = pd.DataFrame()

    # Loop through each stock symbol and its associated DataFrame
    for symbol, data in stock_data.items():
        if 'close' in data.columns and symbol in weights:
            if not isinstance(data.index, pd.DatetimeIndex):
                print(f"Warning: Index for {symbol} is not a datetime index.")
                continue
            
            # Normalize the time series to start at 1
            normalized_series = data['close'] / data['close'].iloc[0]
            # Apply the weight to the normalized time series
            weighted_series = normalized_series * weights[symbol]
            # Add the weighted time series to the portfolio DataFrame
            portfolio_time_series[symbol] = weighted_series
        else:
            print(f"Warning: 'close' column not found or weight missing for {symbol}")

    # Sum the weighted series across all stocks to create the composite portfolio time series
    portfolio_time_series['portfolio'] = portfolio_time_series.sum(axis=1)

    # Plotting
    plt.figure(figsize=(14, 7))
    for column in portfolio_time_series.columns:
        plt.plot(portfolio_time_series.index, portfolio_time_series[column], label=column)
    
    plt.title('Portfolio and Asset Performance')
    plt.xlabel('Date')
    plt.ylabel('Normalized Weighted Value')
    plt.legend()
    plt.grid(True)
    plt.show()

    return portfolio_time_series

In [341]:
uncorrelated_symbols = diversification_through_correlation(all_data, threshold=0.05)

diversified_portfolio = {}
for symbol in tqdm(uncorrelated_symbols):
    diversified_portfolio[symbol] = query_symbol_data(spark, keyspace, table, symbol, start_date, end_date)


100%|██████████| 241/241 [00:54<00:00,  4.45it/s]


In [342]:
favorable_symbols = risk_and_return_analysis(diversified_portfolio)
print("Favorable Stocks:", favorable_symbols)


risk_adjusted_portfolio = {}
for symbol in tqdm(favorable_symbols.index.tolist()):
    risk_adjusted_portfolio[symbol] = query_symbol_data(spark, keyspace, table, symbol, start_date, end_date)


Favorable Stocks:          Return  Volatility  Sharpe Ratio
VSTS   0.867376    0.415928      1.844973
ABVX   1.787438    0.433005      3.897035
NLOP   3.295598    0.805799      3.965749
KGS    0.721756    0.283228      2.195249
CHSN   2.009353    1.900987      1.004401
ASO    0.591099    0.483592      1.015523
JL    31.885760    1.516145     20.964852


100%|██████████| 7/7 [00:01<00:00,  6.25it/s]


In [343]:
weights = optimize_portfolio(risk_adjusted_portfolio, target_return = 0.1)

In [352]:
#portfolio = construct_and_plot_portfolio(risk_adjusted_portfolio, weights)

risk_adjusted_portfolio['VSTS']

,symbol,date,adj_close,close,high,low,open,volume
0,VSTS,2023-10-03 01:00:00,16.60,16.60,17.350000,15.890000,17.010000,5760600
1,VSTS,2023-10-04 01:00:00,16.24,16.24,16.809999,15.880000,16.809999,3647200
2,VSTS,2023-10-05 01:00:00,15.65,15.65,15.900000,14.950000,15.650000,5824400
3,VSTS,2023-10-06 01:00:00,15.36,15.36,15.750000,14.335000,15.150000,5130700
4,VSTS,2023-10-09 01:00:00,15.00,15.00,15.300000,14.550000,15.110000,3149000
...,...,...,...,...,...,...,...,...
78,VSTS,2024-01-25 00:00:00,21.48,21.48,21.510000,20.940001,21.230000,953800
79,VSTS,2024-01-26 00:00:00,21.32,21.32,21.830000,21.290001,21.639999,786600
80,VSTS,2024-01-29 00:00:00,21.76,21.76,21.799999,21.150000,21.320000,1534400
81,VSTS,2024-01-30 00:00:00,21.83,21.83,21.870001,21.440001,21.610001,1210100


In [330]:
# Use the functions
trend = trend_analysis(portfolio)
volatility = volatility_analysis(portfolio)
cyclicity = test_cyclicity(portfolio)

plot_autocorrelation(portfolio)
plot_rolling_statistics(portfolio)
plot_seasonal_decompose(portfolio)

print("Trend Analysis:", trend)
print("Volatility:", volatility)
print(cyclicity)

KeyError: 'close'